In [50]:
# !pip install tiktoken

Model references
================
https://learn.microsoft.com/en-US/azure/ai-services/openai/reference

Tutorial references
===================
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=command-line


In [51]:
import openai
import os
import re
import requests
# import sys
# from num2words import num2words
import os
import pandas as pd
# import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-05-15"

url = openai.api_base + "/openai/deployments?api-version=2023-03-15-preview" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "code-davinci-002",
      "owner": "organization-owner",
      "id": "codex1",
      "status": "succeeded",
      "created_at": 1676872153,
      "updated_at": 1676872153,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "test-davinci-003",
      "status": "succeeded",
      "created_at": 1676949323,
      "updated_at": 1676949323,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-similarity-curie-001",
      "owner": "organization-owner",
      "id": "text-similarity-curie-001",
      "status": "succeeded",
      "created_at": 1676949568,
      "updated_at": 1676949568,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "st

In [67]:
df = pd.read_csv("master.csv") # example: df = pd.read_csv("./data/bill_sum_data.csv")df
df

,videoId,title,description,speaker,text
0,kADcqtPlXdk,Paul Usher,NaN,Fun with IoT &amp; Raspberry Pi 3,so good morning everyone welcome to DVD per an...
1,W3kJx_dsXm8,The Billion Event Challenge,"Event sourcing is popular for many reasons, bu...",Chris Bellew,um thanks for tuning in everyone um i'm going ...
2,W3kJx_dsXm8,The Billion Event Challenge,"Event sourcing is popular for many reasons, bu...",Chris Bellew,per ssd um interestingly the the right threads...
3,gowvZ_EgZ3c,You&#39;re doing TypeScript wrong,TypeScript is a great language with a ton of f...,Ben Lowry,hello hello hello hello welcome welcome back s...
4,JmWLYpofw3g,MicroServices UI Composition,Ever wondered how websites like Amazon were cr...,Hadi Eskandari,Wow full room popular talk last stop for lunch...
...,...,...,...,...,...
250,LKoeAflDXaE,How to Manage Your Ducks,Ever feel like everything is a struggle? Like ...,Amy Kapernick,so next up we have Amy Kaepernick giving her t...
251,BiP_ybxkY7A,The Cat In The Box,"In 1935, Erwin Schrodinger put a (hypothetical...",Chris Ferrie,just all of a sudden got very quiet welcome ba...
252,BiP_ybxkY7A,The Cat In The Box,"In 1935, Erwin Schrodinger put a (hypothetical...",Chris Ferrie,diagrams um yeah and so I did that and it seem...
253,4S2tT8tqc-w,Dr Vivienne Conway,Learn the approach followed in Australia relat...,"Web Accessibility: Responsibilities, Laws and ...",hopefully this afternoon you're in here for a ...


In [68]:
df_bills = df[['text', 'videoId', 'description', 'title', 'speaker']]
df_bills

,text,videoId,description,title,speaker
0,so good morning everyone welcome to DVD per an...,kADcqtPlXdk,NaN,Paul Usher,Fun with IoT &amp; Raspberry Pi 3
1,um thanks for tuning in everyone um i'm going ...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew
2,per ssd um interestingly the the right threads...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew
3,hello hello hello hello welcome welcome back s...,gowvZ_EgZ3c,TypeScript is a great language with a ton of f...,You&#39;re doing TypeScript wrong,Ben Lowry
4,Wow full room popular talk last stop for lunch...,JmWLYpofw3g,Ever wondered how websites like Amazon were cr...,MicroServices UI Composition,Hadi Eskandari
...,...,...,...,...,...
250,so next up we have Amy Kaepernick giving her t...,LKoeAflDXaE,Ever feel like everything is a struggle? Like ...,How to Manage Your Ducks,Amy Kapernick
251,just all of a sudden got very quiet welcome ba...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie
252,diagrams um yeah and so I did that and it seem...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie
253,hopefully this afternoon you're in here for a ...,4S2tT8tqc-w,Learn the approach followed in Australia relat...,Dr Vivienne Conway,"Web Accessibility: Responsibilities, Laws and ..."


In [69]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))

In [70]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

255

In [71]:
df_bills

,text,videoId,description,title,speaker,n_tokens
0,so good morning everyone welcome to DVD per an...,kADcqtPlXdk,NaN,Paul Usher,Fun with IoT &amp; Raspberry Pi 3,5992
1,um thanks for tuning in everyone um i'm going ...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew,6876
2,per ssd um interestingly the the right threads...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew,1567
3,hello hello hello hello welcome welcome back s...,gowvZ_EgZ3c,TypeScript is a great language with a ton of f...,You&#39;re doing TypeScript wrong,Ben Lowry,2491
4,Wow full room popular talk last stop for lunch...,JmWLYpofw3g,Ever wondered how websites like Amazon were cr...,MicroServices UI Composition,Hadi Eskandari,4786
...,...,...,...,...,...,...
250,so next up we have Amy Kaepernick giving her t...,LKoeAflDXaE,Ever feel like everything is a struggle? Like ...,How to Manage Your Ducks,Amy Kapernick,2992
251,just all of a sudden got very quiet welcome ba...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie,6668
252,diagrams um yeah and so I did that and it seem...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie,985
253,hopefully this afternoon you're in here for a ...,4S2tT8tqc-w,Learn the approach followed in Australia relat...,Dr Vivienne Conway,"Web Accessibility: Responsibilities, Laws and ...",6456


In [72]:
print(df_bills.text[2])

sample_encode = tokenizer.encode(df_bills.text[2]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

per ssd um interestingly the the right threads there um are starting to get a bit bit busy um so maybe i could bump those right threads up and see if i get the same same uh boost like i did with the read threads uh yeah i didn't really maybe i could do that more but it appears as though it's not really taking too much time there beyond what the read threads and processing threads are doing so i might be reaching diminishing returns with the two disks um so that probably is the most i could get to 34 seconds on my home machine so what do i do i could go out and buy more disks that appears to be the magic um the the cpu i don't appear to be fully utilizing that so i can go to ple and get sort of four more drives um or i could call up jeff bezos and say hey one of your machines can i can i borrow one of those um one of those big beefy ones and he said yeah i'm gonna be in space i don't need these you can you can have these um so yeah go for your life so i looked up the instances and i fou

[b'per',
 b' ss',
 b'd',
 b' um',
 b' interesting',
 b'ly',
 b' the',
 b' the',
 b' right',
 b' threads',
 b' there',
 b' um',
 b' are',
 b' starting',
 b' to',
 b' get',
 b' a',
 b' bit',
 b' bit',
 b' busy',
 b' um',
 b' so',
 b' maybe',
 b' i',
 b' could',
 b' bump',
 b' those',
 b' right',
 b' threads',
 b' up',
 b' and',
 b' see',
 b' if',
 b' i',
 b' get',
 b' the',
 b' same',
 b' same',
 b' uh',
 b' boost',
 b' like',
 b' i',
 b' did',
 b' with',
 b' the',
 b' read',
 b' threads',
 b' uh',
 b' yeah',
 b' i',
 b' didn',
 b"'t",
 b' really',
 b' maybe',
 b' i',
 b' could',
 b' do',
 b' that',
 b' more',
 b' but',
 b' it',
 b' appears',
 b' as',
 b' though',
 b' it',
 b"'s",
 b' not',
 b' really',
 b' taking',
 b' too',
 b' much',
 b' time',
 b' there',
 b' beyond',
 b' what',
 b' the',
 b' read',
 b' threads',
 b' and',
 b' processing',
 b' threads',
 b' are',
 b' doing',
 b' so',
 b' i',
 b' might',
 b' be',
 b' reaching',
 b' diminishing',
 b' returns',
 b' with',
 b' the',
 b' 

In [73]:
len(decode)

1567

In [74]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [75]:
df_bills

,text,videoId,description,title,speaker,n_tokens,ada_v2
0,so good morning everyone welcome to DVD per an...,kADcqtPlXdk,NaN,Paul Usher,Fun with IoT &amp; Raspberry Pi 3,5992,"[0.006476980168372393, -0.003036953043192625, ..."
1,um thanks for tuning in everyone um i'm going ...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew,6876,"[-0.03605175390839577, 0.0006079665618017316, ..."
2,per ssd um interestingly the the right threads...,W3kJx_dsXm8,"Event sourcing is popular for many reasons, bu...",The Billion Event Challenge,Chris Bellew,1567,"[-0.02873470075428486, 0.003970840480178595, 0..."
3,hello hello hello hello welcome welcome back s...,gowvZ_EgZ3c,TypeScript is a great language with a ton of f...,You&#39;re doing TypeScript wrong,Ben Lowry,2491,"[0.0005881282850168645, 0.011359688825905323, ..."
4,Wow full room popular talk last stop for lunch...,JmWLYpofw3g,Ever wondered how websites like Amazon were cr...,MicroServices UI Composition,Hadi Eskandari,4786,"[-0.003973057027906179, 0.004469688981771469, ..."
...,...,...,...,...,...,...,...
250,so next up we have Amy Kaepernick giving her t...,LKoeAflDXaE,Ever feel like everything is a struggle? Like ...,How to Manage Your Ducks,Amy Kapernick,2992,"[-0.003937470261007547, -0.011479425244033337,..."
251,just all of a sudden got very quiet welcome ba...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie,6668,"[0.011354740709066391, -0.0001864806399680674,..."
252,diagrams um yeah and so I did that and it seem...,BiP_ybxkY7A,"In 1935, Erwin Schrodinger put a (hypothetical...",The Cat In The Box,Chris Ferrie,985,"[0.007196051999926567, -0.004988945554941893, ..."
253,hopefully this afternoon you're in here for a ...,4S2tT8tqc-w,Learn the approach followed in Australia relat...,Dr Vivienne Conway,"Web Accessibility: Responsibilities, Laws and ...",6456,"[0.001312287524342537, 0.009910122491419315, 0..."


In [76]:
# Persist the embeddings to disk as a csv file
df_bills.to_csv("master_embeddings.csv", index=False)

In [65]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "sessions about vs code", top_n=4)

,text,videoId,description,title,speaker,n_tokens,ada_v2,similarities
8,to CSS and that's telling me this is uh banner...,0gZO79pGBow,"VS Code is hot, there's no doubt about it bein...",Unleash the Power of VS Code,Aaron Powell,2936,"[-0.014254079200327396, 0.01091174315661192, 0...",0.760936
7,foreign okay well thank you everyone for comin...,0gZO79pGBow,"VS Code is hot, there's no doubt about it bein...",Unleash the Power of VS Code,Aaron Powell,7103,"[0.013355430215597153, 0.0017934730276465416, ...",0.759769
4,thing is there's pretty much no protection fro...,2KjEBFAVgoU,NaN,NaN,Mari (Red Gum) Stream,6654,"[-0.009004206396639347, -0.0067765843123197556...",0.759614
3,he's paid off his hex debt something i'm sure ...,2KjEBFAVgoU,NaN,NaN,Mari (Red Gum) Stream,6531,"[0.005740201100707054, 0.007738327607512474, 0...",0.752595


In [63]:
res["text"][7]

"foreign okay well thank you everyone for coming back to jacquel Negal again up next we have Aaron who's going to be talking about unleashing the power of Visual Studio code probably one of the more industry changing editors of recent times I would say so please everyone put your hands together to welcome Aaron to the stage thanks for having me um as mentioned my name is Aaron I work for Microsoft as part of the cloud advocacy team I've been doing software development and web development for longer that I'm going to admit to or that in a session that's being recorded and streamed out but I I I've used a lot of editors in my time like I I got started with like pico and um and stuff like that emacs on on Linux machines when I was uh first at Uni and then obviously have graduated to more complex editors but yeah like vs code is it's been huge like in like the way that has changed how people think about you know Simple Text editors because at the end of the day that's that's what it is and